In [1]:
import sys
sys.path.append('/Users/ianleefmans/Desktop/stock-sentiment/utils')
from data import Database, ScrapeWSB, Stock
from datahelper import PostDataset, CommentDataset, get_indices
from torch.utils.data import DataLoader
from inference import RunInference
from models import FineTuneBaseModel, FineTuneClassifier
%cd /Users/ianleefmans/Desktop/stock-sentiment

/Users/ianleefmans/Desktop/stock-sentiment


In [2]:
import transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification

### DB Queries

In [4]:

db = Database()
db.use_database('DB1')

print(db.query('show tables;'))

connection established
[(datetime.datetime(2021, 10, 6, 18, 26, 27),)]
[('COMMENTS',), ('POSTS',), ('STOCKS',)]


In [7]:
db.query("SELECT * FROM STOCKS WHERE STOCK_ID='AMC' AND LAST_SCRAPED >= DATE_SUB(NOW(),INTERVAL 55 MINUTE)")

[('AMC', datetime.datetime(2021, 6, 6, 22, 15, 44))]

In [16]:
db.query("SELECT POST_ID FROM POSTS WHERE STOCK_ID='{}' AND LAST_SCRAPED >= DATE_SUB((SELECT LAST_SCRAPED FROM STOCKS WHERE STOCK_ID = '{}'), INTERVAL {} HOUR);".format('BB', 'BB', 6))



[('nx2ihx',),
 ('nxcb5h',),
 ('nxp5r6',),
 ('nxtjj4',),
 ('nxv9jr',),
 ('nxx9sz',),
 ('ny01wx',),
 ('nz6tgd',),
 ('nzf0mj',),
 ('nzjcfg',)]

In [86]:
db.query("SELECT POST_ID FROM POSTS WHERE STOCK_ID='{}' AND LAST_SCRAPED >= DATE_SUB((SELECT LAST_SCRAPED FROM STOCKS WHERE STOCK_ID = '{}'),INTERVAL {} HOUR);".format('AAPL','AAPL',6))



[('nv0h06',)]

In [85]:
db.query("SELECT LAST_SCRAPED FROM POSTS WHERE LAST_SCRAPED >= DATE_SUB((SELECT LAST_SCRAPED FROM STOCKS WHERE STOCK_ID = 'AAPL'),INTERVAL 6 HOUR);")
         
         
         


[(datetime.datetime(2021, 6, 14, 18, 21, 27),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 43),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 43),),
 (datetime.datetime(2021, 6, 14, 14, 27, 43),),
 (datetime.datetime(2021, 6, 14, 14, 27, 43),)]

### Running Inference

In [2]:
indices = get_indices("GME", inference=True, scrape_time=6)

connection established
[(datetime.datetime(2021, 10, 6, 19, 48, 59),)]


In [3]:
post_train = PostDataset(512, indices['post_ids'])



connection established
[(datetime.datetime(2021, 10, 6, 19, 49, 1),)]


In [4]:
post_trainloader = DataLoader(
    dataset=post_train, 
    batch_size=3, 
    num_workers=0,
    shuffle=True
    )

In [5]:
sample = iter(post_trainloader).next()

In [6]:
sample


{'post': ['GIANT $RUN LOVING APE BALLS OUT.....$BB $AMC $GME',
  'Jim Cramer WSB FOMO- GME and AMC bull!',
  'Daily option statistics for AMC, CLNE, BB, WISH, GME, WKHS, CLOV, PLTR, CLF, and others.'],
 'post_input_ids': tensor([[ 101,  144, 9984,  ...,    0,    0,    0],
         [ 101, 3104,  140,  ...,    0,    0,    0],
         [ 101, 5732, 5146,  ...,    0,    0,    0]]),
 'post_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'target': tensor([-1, -1, -1]),
 'date': tensor([1.6240e+09, 1.6240e+09, 1.6240e+09], dtype=torch.float64)}

In [7]:
model = FineTuneClassifier()

out = model(input_ids=sample['post_input_ids'], attention_masks=sample['post_attention_mask'])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:
out

tensor([[0.5016, 0.4984],
        [0.5107, 0.4893],
        [0.4868, 0.5132]], grad_fn=<SoftmaxBackward>)

### Stock Predictions

In [2]:
from inference import RunInference

run =RunInference('GME')
output = run.evaluate()

connection established
[(datetime.datetime(2021, 10, 9, 17, 3, 50),)]
connection established
[(datetime.datetime(2021, 10, 9, 17, 3, 52),)]


Determining Sentiment From Posts:   0%|          | 0/1 [00:00<?, ?it/s]

connection established
[(datetime.datetime(2021, 10, 9, 17, 3, 54),)]


Determining Sentiment From Comments: 100%|██████████| 1/1 [00:08<00:00,  8.72s/it]


In [3]:
output

{'avg_post_probs': tensor([3.6181e-04, 9.9964e-01]),
 'avg_comment_probs': tensor([0.0803, 0.9197]),
 'all_post_probs': tensor([[6.6488e-04, 9.9934e-01],
         [1.8537e-04, 9.9981e-01],
         [5.7578e-04, 9.9942e-01],
         [2.1131e-04, 9.9979e-01],
         [1.7168e-04, 9.9983e-01]]),
 'all_comment_probs': tensor([[4.0072e-01, 5.9928e-01],
         [1.6010e-04, 9.9984e-01],
         [3.3146e-04, 9.9967e-01],
         [1.6648e-04, 9.9983e-01],
         [1.9659e-04, 9.9980e-01]]),
 'all_post_dates': tensor([1.6240e+09, 1.6240e+09, 1.6239e+09, 1.6239e+09, 1.6240e+09],
        dtype=torch.float64),
 'all_comment_dates': tensor([1.6240e+09, 1.6239e+09, 1.6240e+09, 1.6240e+09, 1.6239e+09],
        dtype=torch.float64),
 'max_post_prob': 0.9998283386230469,
 'max_comment_prob': 0.9998399019241333,
 'max_post': 'Jim Cramer WSB FOMO- GME and AMC bull!',
 'max_comment': '#BB GANG 🦍🦍🦍',
 'min_post_prob': 0.9993351101875305,
 'min_comment_prob': 0.5992765426635742,
 'min_post': 'Daily op

In [3]:
from forecast import Forecast

get_stock = Forecast("GME", output)

100%|██████████| 786/786 [00:01<00:00, 481.74it/s]


In [4]:
get_stock.assign_sentiment()
get_stock.stock_data

,close,high,low,open,status,timestamp,volume,highlow_percent,highlow_percent_t1,sentiment
0,168.78,169.130,168.78,169.000,ok,2021-10-06 13:06:00,1973,0.997931,NaN,0.799546
1,168.60,168.775,168.60,168.775,ok,2021-10-06 13:07:00,1470,0.998963,NaN,0.799546
2,168.80,168.800,168.50,168.640,ok,2021-10-06 13:09:00,4365,0.998223,NaN,0.799546
3,168.55,168.550,168.55,168.550,ok,2021-10-06 13:11:00,1236,1.000000,NaN,0.799546
4,168.81,168.810,168.81,168.810,ok,2021-10-06 13:12:00,248,1.000000,NaN,0.799546
...,...,...,...,...,...,...,...,...,...,...
781,172.45,172.690,172.45,172.690,ok,2021-10-08 16:16:00,375,0.998610,0.999004,0.799546
782,172.45,172.450,172.45,172.450,ok,2021-10-08 16:17:00,198,1.000000,0.999004,0.799546
783,172.95,172.950,172.95,172.950,ok,2021-10-08 17:11:00,179,1.000000,0.999004,0.799546
784,172.72,172.720,172.72,172.720,ok,2021-10-08 17:34:00,148,1.000000,0.999004,0.799546


In [5]:

table = get_stock.sentiment
table

,date,sentiment
0,2021-06-17 03:20:00,0.999614
1,2021-06-17 06:06:40,0.999814
2,2021-06-17 22:46:40,0.999831
3,2021-06-18 04:20:00,0.999502
4,2021-06-18 07:06:40,0.799546


In [4]:
results = get_stock.arima()

In [6]:
print(results.summary())

                               SARIMAX Results                                
Dep. Variable:                  close   No. Observations:                 1090
Model:                 ARIMA(1, 0, 0)   Log Likelihood                -406.430
Date:                Wed, 06 Oct 2021   AIC                            820.860
Time:                        13:56:50   BIC                            840.836
Sample:                             0   HQIC                           828.421
                               - 1090                                         
Covariance Type:                  opg                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             139.7038      0.002   5.63e+04      0.000     139.699     139.709
highlow_percent     3.9948      2.846      1.403      0.160      -1.584       9.574
ar.L1               1.0000   1.3